In [ ]:
!pip install transformers

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_excel("ikn_tweet_2.xlsx")
data.head(5)

,created_at,full_text,username,tweet_url
0,Tue Apr 30 23:56:33 +0000 2024,Tinjau Pembangunan Fasilitas Pendukung Parekra...,okezonenews,https://twitter.com/okezonenews/status/1785458...
1,Tue Apr 30 23:56:22 +0000 2024,Iyalah presiden yang utang duit buat NEGARA Da...,ajayharau,https://twitter.com/ajayharau/status/178545820...
2,Tue Apr 30 23:55:45 +0000 2024,"Nama akunnya IKN Nusantara"" Ngomong ""pemerataa...",abidzara___,https://twitter.com/abidzara___/status/1785458...
3,Tue Apr 30 23:52:00 +0000 2024,"Di bumi Kalimantan Timur, sebuah transformasi ...",nusabalicom,https://twitter.com/nusabalicom/status/1785457...
4,Tue Apr 30 23:50:51 +0000 2024,Presiden Jokowi aja investor Singapura membagu...,Inocantiksekali,https://twitter.com/Inocantiksekali/status/178...


In [ ]:
from googletrans.client import Translator
translator = Translator()

from transformers import pipeline
sentiment_classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def clean_tweet(tweet):
  return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", tweet).split())

In [ ]:
data['label_bert'] = data['full_text'].str.encode('ascii','ignore').apply(translator.translate, dest='en')
data['label_bert'] = data['label_bert'].apply(getattr, args=('text',))
data['label_bert'] = data.apply(lambda x: clean_tweet(x['label_bert']), axis=1)

In [ ]:
df = (
    data
    .assign(sentiment = lambda x: x['label_bert'].apply(lambda s: sentiment_classifier(s)))
    .assign(
        label_bert = lambda x: x['sentiment'].apply(lambda s:(s[0]['label'])),
        score = lambda x: x['sentiment'].apply(lambda s: (s[0]['score']))
    )
  )

In [ ]:
df.head(5)

,created_at,full_text,username,tweet_url,label_bert,sentiment,score
0,Tue Apr 30 23:56:33 +0000 2024,Tinjau Pembangunan Fasilitas Pendukung Parekra...,okezonenews,https://twitter.com/okezonenews/status/1785458...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.967255651950...",0.967256
1,Tue Apr 30 23:56:22 +0000 2024,Iyalah presiden yang utang duit buat NEGARA Da...,ajayharau,https://twitter.com/ajayharau/status/178545820...,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.967068672180...",0.967069
2,Tue Apr 30 23:55:45 +0000 2024,"Nama akunnya IKN Nusantara"" Ngomong ""pemerataa...",abidzara___,https://twitter.com/abidzara___/status/1785458...,NEGATIVE,"[{'label': 'NEGATIVE', 'score': 0.948749542236...",0.948750
3,Tue Apr 30 23:52:00 +0000 2024,"Di bumi Kalimantan Timur, sebuah transformasi ...",nusabalicom,https://twitter.com/nusabalicom/status/1785457...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.999265134334...",0.999265
4,Tue Apr 30 23:50:51 +0000 2024,Presiden Jokowi aja investor Singapura membagu...,Inocantiksekali,https://twitter.com/Inocantiksekali/status/178...,POSITIVE,"[{'label': 'POSITIVE', 'score': 0.991879403591...",0.991879


In [ ]:
df.label_bert.value_counts()

label_bert
NEGATIVE    85
POSITIVE    39
Name: count, dtype: int64

In [ ]:
dt_bert = df[['full_text','label_bert']]
dt_bert

,full_text,label_bert
0,Tinjau Pembangunan Fasilitas Pendukung Parekra...,POSITIVE
1,Iyalah presiden yang utang duit buat NEGARA Da...,NEGATIVE
2,"Nama akunnya IKN Nusantara"" Ngomong ""pemerataa...",NEGATIVE
3,"Di bumi Kalimantan Timur, sebuah transformasi ...",POSITIVE
4,Presiden Jokowi aja investor Singapura membagu...,POSITIVE
...,...,...
119,Bayar pajak aja susah Tp sok sok an bangun IKN,NEGATIVE
120,Terus ngapain bangun IKN kalo tau kita miskin?...,NEGATIVE
121,"Kan tau miskin, ngapain bangun IKN?",NEGATIVE
122,"Lagian yg di bangun ikn, transportasinya malah...",NEGATIVE


In [ ]:
dt_aktual = pd.read_excel("ikn_tweet_2_aktual.xlsx")
dt_aktual.head(5)

,created_at,full_text,username,tweet_url,label
0,Tue Apr 30 23:56:33 +0000 2024,Tinjau Pembangunan Fasilitas Pendukung Parekra...,okezonenews,https://twitter.com/okezonenews/status/1785458...,POSITIVE
1,Tue Apr 30 23:56:22 +0000 2024,Iyalah presiden yang utang duit buat NEGARA Da...,ajayharau,https://twitter.com/ajayharau/status/178545820...,NEGATIVE
2,Tue Apr 30 23:55:45 +0000 2024,"Nama akunnya IKN Nusantara"" Ngomong ""pemerataa...",abidzara___,https://twitter.com/abidzara___/status/1785458...,NEGATIVE
3,Tue Apr 30 23:52:00 +0000 2024,"Di bumi Kalimantan Timur, sebuah transformasi ...",nusabalicom,https://twitter.com/nusabalicom/status/1785457...,POSITIVE
4,Tue Apr 30 23:50:51 +0000 2024,Presiden Jokowi aja investor Singapura membagu...,Inocantiksekali,https://twitter.com/Inocantiksekali/status/178...,POSITIVE


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score


In [ ]:
dt_perbandingan = pd.concat([dt_aktual[['full_text','label']], dt_bert[['label_bert']]], axis=1)
dt_perbandingan

,full_text,label,label_bert
0,Tinjau Pembangunan Fasilitas Pendukung Parekra...,POSITIVE,POSITIVE
1,Iyalah presiden yang utang duit buat NEGARA Da...,NEGATIVE,NEGATIVE
2,"Nama akunnya IKN Nusantara"" Ngomong ""pemerataa...",NEGATIVE,NEGATIVE
3,"Di bumi Kalimantan Timur, sebuah transformasi ...",POSITIVE,POSITIVE
4,Presiden Jokowi aja investor Singapura membagu...,POSITIVE,POSITIVE
...,...,...,...
119,Bayar pajak aja susah Tp sok sok an bangun IKN,NEGATIVE,NEGATIVE
120,Terus ngapain bangun IKN kalo tau kita miskin?...,NEGATIVE,NEGATIVE
121,"Kan tau miskin, ngapain bangun IKN?",NEGATIVE,NEGATIVE
122,"Lagian yg di bangun ikn, transportasinya malah...",NEGATIVE,NEGATIVE


In [ ]:
dt_perbandingan.to_excel('dt_perbandingan.xlsx', index=False)

In [ ]:
akurasi = accuracy_score(dt_aktual['label'], dt_bert['label_bert'])
print("Akurasi:", akurasi)

Akurasi: 0.7016129032258065


### **Distil Bert**